In [2]:
# ensure the follow packages are downloaded before beginning
# pip install scikit-image
# pip install opencv-python 

In [24]:
import numpy as np
import matplotlib.image as mimg
import matplotlib.pyplot as plt
from skimage import feature # to extract the features
from sklearn import svm # svm as classifer
import joblib # to save the model 
import os
import pandas as pd

import cv2  # the image processing library
import matplotlib.pyplot as plt  
   

In [25]:
# creating empty np arrays for training and testing

length = len(next(os.walk('orl_face'))[1]) # total training samples (number of people)
no_samples = 7 #set the number of images trained for each person (max 10)

train_data=np.zeros((no_samples*length,280))
train_label=np.zeros((no_samples*length))
count=-1
#plt.figure(1)
#plt.ion()


# training loop

for i in range(1,length+1):
    for j in range(1,no_samples+1):
        #plt.cla()
        count=count+1
        path = 'orl_face/u%d/%d.png'%(i,j)
        im = mimg.imread(path)
        # feature extraction 
        feat,hog_image = feature.hog(im,orientations=8,pixels_per_cell=(16,16),
                                     visualize=True,block_norm='L2-Hys',
                                     cells_per_block=(1,1))
        
        train_data[count,:]=feat.reshape(1,-1)
        train_label[count]=i
        # histogram of gradient features(hog)
        #plt.subplot(2,1,1)
        #plt.imshow(im,cmap='gray') # displaying training images
        #plt.subplot(2,1,2)
        #plt.imshow(hog_image,cmap='gray')
        #plt.pause(0.3)

# model creation
svm_model = svm.SVC(kernel='poly',gamma='scale')

# train the model
svm_model = svm_model.fit(train_data,train_label)

# save the model
joblib.dump(svm_model,'svm_train_model_face.pkl')

print('training done')

training done


In [26]:

# you can reload the model directly instead of fitting again 

joblib.load('svm_train_model_face.pkl')


SVC(kernel='poly')

In [23]:
# Capturing your images for training!


num_of_sample = 10 # set number of samples

# make a new folder 
if not os.path.exists('orl_face/u%d'%(length + 1)):
    os.makedirs('orl_face/u%d'%(length + 1))
    
vid = cv2.VideoCapture(0) # to open the camera

# haar cascade for frontal face is the algorithm to detect the face in image
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
iter1=0

while(iter1<num_of_sample):
    r,frame = vid.read();# capture a single frame
    frame = cv2.resize(frame,(640,480)) # resizig the frame
    im1 = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY) 
    face=face_cascade.detectMultiScale(im1, scaleFactor =1.3, minNeighbors=4, minSize= (30,30))
    for x,y,w,h in face:
        cv2.rectangle(frame,(x,y),(x+w,y+h),[0,0,255],4) 
        iter1=iter1+1
        im_f = im1[y:y+h,x:x+w] #cropping of the face from image
        im_f = cv2.resize(im_f,(112,92)) #orl face matching size
        cv2.putText(frame,'sample no.'+str(iter1),(x,y), cv2.FONT_ITALIC, 1,
                   (255,0,255),2,cv2.LINE_AA)
        path2 = 'orl_face/u%d/%d.png'%(length + 1,iter1) # path to save the image directly to a new folder in the training data
        cv2.imwrite(path2,im_f) # to save the image 
        
    cv2.imshow('frame',frame)# display
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [19]:
# testing on images captured in real time on video, with the predicted label printed on the left of the red box

import cv2 
import matplotlib.pyplot as plt   # for the plotting

num_of_sample = 100    
vid = cv2.VideoCapture(0) # to open the camera

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
iter1=0

while(iter1<num_of_sample):
    r,frame = vid.read();# capture a single frame
    
    im1 = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)# gray scale conversion of color image
    face=face_cascade.detectMultiScale(im1, scaleFactor =1.3, minNeighbors=4, minSize= (30,30))
    eye = eye_cascade.detectMultiScale(im1)
    for x,y,w,h in face:
        cv2.rectangle(frame,(x,y),(x+w,y+h),[0,0,255],4)
        iter1=iter1+1
        im_f= im1[y:y+h,x:x+w] #cropping of the face from image
        im_f = cv2.resize(im_f,(112,92)) #orl face matching size
        feat,hog_image = feature.hog(im,orientations=8,pixels_per_cell=(16,16),
                                     visualize=True,block_norm='L2-Hys',
                                     cells_per_block=(1,1))

        test =feat.reshape(1,-1)
        # make prediction
        result = str(svm_model.predict(test)[0])
        
        # put prediction label next to the image
        cv2.putText(frame,result,(x,y), cv2.FONT_ITALIC, 1,
                   (255,0,255),2,cv2.LINE_AA)
    for ex,ey,ew,eh in eye:
        cv2.rectangle(frame, (ex,ey), (ex+ew, ey+eh), (0,225,0),2)
    cv2.imshow('frame',frame)# display
    if cv2.waitKey(1) and 0xFF == ord('q'):
             break
vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


# and there you have it! you can define a dictionary if you wish for the names to be printed instead of labels

-1